In [1]:
import yfinance as yf
import pandas as pd
import awswrangler as wr
import boto3

In [2]:
# nasdak_100
tickers = [
    "ADBE","AMD","ABNB","GOOGL","GOOG","AMZN","AEP","AMGN","ADI","AAPL",
    "AMAT","APP","ARM","ASML","AZN","TEAM","ADSK","ADP","AXON","BKR",
    "BIIB","BKNG","AVGO","CDNS","CDW","CHTR","CTAS","CSCO","CCEP","CTSH",
    "CMCSA","CEG","CPRT","CSGP","COST","CRWD","CSX","DDOG","DXCM","FANG",
    "DASH","EA","EXC","FAST","FTNT","GEHC","GILD","GFS","HON","IDXX",
    "INTC","INTU","ISRG","KDP","KLAC","KHC","LRCX","LIN","LULU","MAR",
    "MRVL","MELI","META","MCHP","MU","MSFT","MSTR","MDLZ","MNST","NFLX",
    "NVDA","NXPI","ORLY","ODFL","ON","PCAR","PLTR","PANW","PAYX","PYPL",
    "PDD","PEP","QCOM","REGN","ROP","ROST","SHOP","SBUX","SNPS","TMUS",
    "TTWO","TSLA","TXN","TRI","TTD","VRSK","VRTX","WBD","WDAY","XEL","ZS"
]

data = yf.download(tickers, period='3mo')

C:\Users\gabriel\AppData\Local\Temp\ipykernel_7956\201727266.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period='3mo')
[*********************100%***********************]  101 of 101 completed


In [ ]:
df_reset = data.copy().reset_index()  # transforma 'Date' em coluna


dfs = []
for ticker in data.columns.levels[1]:  
    temp = pd.DataFrame({
        "Date": df_reset["Date"],
        "Ticker": ticker,
        "Open": df_reset[("Open", ticker)],
        "High": df_reset[("High", ticker)],
        "Low": df_reset[("Low", ticker)],
        "Close": df_reset[("Close", ticker)],
        "Volume": df_reset[("Volume", ticker)],
    })
    dfs.append(temp)

df_final = pd.concat(dfs, ignore_index=True)

df_final.to_parquet(
    "nasdaq_data_parquet",
    engine="pyarrow",
    partition_cols=["Date"],
    index=False
)

print(df_final)

           Date Ticker        Open        High         Low       Close  \
0    2025-07-07   AAPL  212.438891  215.984870  208.563300  209.711990   
1    2025-07-08   AAPL  209.861836  211.190315  208.213698  209.771927   
2    2025-07-09   AAPL  209.292475  211.090437  206.985096  210.900650   
3    2025-07-10   AAPL  210.271350  213.237984  209.791898  212.169205   
4    2025-07-11   AAPL  210.331296  211.889525  209.622094  210.920624   
...         ...    ...         ...         ...         ...         ...   
6459 2025-09-29     ZS  297.109985  300.809998  293.510010  296.899994   
6460 2025-09-30     ZS  295.839996  300.209991  291.570007  299.660004   
6461 2025-10-01     ZS  298.609985  305.910004  297.135010  304.529999   
6462 2025-10-02     ZS  306.000000  307.790009  302.149994  307.579987   
6463 2025-10-03     ZS  308.070007  309.790009  301.709991  305.410004   

        Volume  
0     50229000  
1     42848900  
2     48749400  
3     44443600  
4     39765800  
...      

In [4]:
# Suponha que df_final já tenha a coluna 'Date'
df_final['Date'] = pd.to_datetime(df_final['Date'])
df_final['ano'] = df_final['Date'].dt.year
df_final['mes'] = df_final['Date'].dt.month
df_final['dia'] = df_final['Date'].dt.day

s3_path = "s3://dados-gabriel-espanhol/raw/"

# Iterar sobre cada partição
for (ano, mes, dia), df_part in df_final.groupby(["ano", "mes", "dia"]):
    # Caminho no formato que você pediu
    path_part = f"{s3_path}{ano}/{mes:02d}/{dia:02d}/data.parquet"

    # escreve só aquela partição (sem mode!)
    wr.s3.to_parquet(
        df=df_part,
        path=path_part,
        dataset=False  # salva como arquivo simples
    )
  
    print(f"Partição {ano}/{mes}/{dia} enviada em {path_part}")
    # time.sleep(5)  # delay entre as partições


Partição 2025/7/7 enviada em s3://dados-gabriel-espanhol/raw/2025/07/07/data.parquet
Partição 2025/7/8 enviada em s3://dados-gabriel-espanhol/raw/2025/07/08/data.parquet
Partição 2025/7/9 enviada em s3://dados-gabriel-espanhol/raw/2025/07/09/data.parquet
Partição 2025/7/10 enviada em s3://dados-gabriel-espanhol/raw/2025/07/10/data.parquet
Partição 2025/7/11 enviada em s3://dados-gabriel-espanhol/raw/2025/07/11/data.parquet
Partição 2025/7/14 enviada em s3://dados-gabriel-espanhol/raw/2025/07/14/data.parquet
Partição 2025/7/15 enviada em s3://dados-gabriel-espanhol/raw/2025/07/15/data.parquet
Partição 2025/7/16 enviada em s3://dados-gabriel-espanhol/raw/2025/07/16/data.parquet
Partição 2025/7/17 enviada em s3://dados-gabriel-espanhol/raw/2025/07/17/data.parquet
Partição 2025/7/18 enviada em s3://dados-gabriel-espanhol/raw/2025/07/18/data.parquet
Partição 2025/7/21 enviada em s3://dados-gabriel-espanhol/raw/2025/07/21/data.parquet
Partição 2025/7/22 enviada em s3://dados-gabriel-espanhol

In [ ]:
s3 = boto3.client("s3")

bucket = "dados-gabriel-espanhol"
chave = "start_job.txt"

conteudo = "start glue job"

# Upload a partir de string
s3.put_object(Bucket=bucket, Key=chave, Body=conteudo.encode("utf-8"))

print(f"Envia o arquivo para s3://{bucket}/{chave}")

✅ Arquivo enviado direto da memória para s3://dados-gabriel-espanhol/start_job.txt
